In [1]:
# import libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
import pickle
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import os

In [2]:
# set pandas visualization options

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# read train preprocessed

base_path=os.path.abspath(os.getcwd())

train=pd.read_csv(base_path+"\\data-scientist-technical-test-main\\data\\auto-insurance-fall-2017\\train_auto_preproc.csv", sep=",", header=0)

In [4]:
#drop extra column

drop_col = ["Unnamed: 0"]
target_col=["TARGET_FLAG"]

train.drop(columns=drop_col, axis=1, inplace=True)

# divide X,y
X=train.drop(columns=target_col, axis=1)
y=train[target_col].values

In [5]:
# create 5-folds

n_splits = 5

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=18, stratify=y)

kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=18)

## logistic regression

In [6]:
# create pipeline object to standardize data for LR

pipe = Pipeline(steps=[('standard_scaler', StandardScaler()),
                       ('LR', LogisticRegression(random_state=18,
                                                 max_iter=200))])

In [7]:
param_grid = dict(LR__penalty = ['l1', 'l2', 'elasticnet'],
                  LR__C = np.logspace(-2, 3, 6))

grid = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, 
                    verbose=1,
                    scoring='f1',
                    return_train_score=True)
grid.fit(X_train, y_train)
print(grid.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

0.5029242287860396


C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, se

In [8]:
#show cv results
cv_results = pd.DataFrame(grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_LR__C',
            'param_LR__penalty', 
            'mean_test_score',
            'std_test_score',
            'rank_test_score']].head(5)

,param_LR__C,param_LR__penalty,mean_test_score,std_test_score,rank_test_score
13,100.0,l2,0.502924,0.023478,1
16,1000.0,l2,0.502924,0.023478,1
10,10.0,l2,0.502392,0.023222,3
7,1.0,l2,0.502035,0.023958,4
4,0.1,l2,0.498298,0.023587,5


In [9]:
C=cv_results[['param_LR__C', 'param_LR__penalty']].iloc[0,:].values[0]
penalty=cv_results[['param_LR__C', 'param_LR__penalty']].iloc[0,:].values[1]
print(C, penalty)

100.0 l2


In [10]:
ss=StandardScaler()
X_transf=ss.fit_transform(X_train)
lr=LogisticRegression(random_state=18,
                      C=C,
                      penalty=penalty).fit(X_transf,y_train)

C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# f1 prediction on validation set
ss=StandardScaler()
X_val_transf=ss.fit_transform(X_val)
y_pred = lr.predict(X_val_transf)
f1_score(y_val, y_pred)

0.49715909090909105

In [12]:
# save LR model
pickle.dump(lr, open(base_path+"\\data-scientist-technical-test-main\\data\\auto-insurance-fall-2017\\models\\LR.pkl", 'wb'))

## random forest

In [13]:
# create pipeline object to standardize data for RF

pipe = Pipeline(steps=[('RFC', RandomForestClassifier())])

param_grid = dict(RFC__max_depth = [5,7,9],
                  RFC__n_estimators = [25,50,75,100])

grid = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, 
                    verbose=1,
                    scoring='f1',
                    return_train_score=True)
grid.fit(X_train, y_train)
print(grid.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was ex

C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was ex

C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


0.39219152241766697


C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [14]:
#show cv results
cv_results = pd.DataFrame(grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_RFC__max_depth',
            'param_RFC__n_estimators', 
            'mean_test_score',
            'std_test_score',
            'rank_test_score']].head(5)

,param_RFC__max_depth,param_RFC__n_estimators,mean_test_score,std_test_score,rank_test_score
8,9,25,0.392192,0.027140,1
11,9,100,0.389934,0.021947,2
9,9,50,0.388721,0.018148,3
10,9,75,0.383377,0.017768,4
4,7,25,0.341544,0.022865,5


In [15]:
max_depth=cv_results[['param_RFC__max_depth', 'param_RFC__n_estimators']].iloc[0,:].values[0]
n_estimators=cv_results[['param_RFC__max_depth', 'param_RFC__n_estimators']].iloc[0,:].values[1]
print(max_depth, n_estimators)

9 25


In [16]:
rf=RandomForestClassifier(max_depth=max_depth,
                          n_estimators=n_estimators,
                          random_state=18).fit(X_train, y_train)

C:\Users\11622563\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# f1 prediction on validation set

y_pred = rf.predict(X_val)
f1_score(y_val, y_pred)

0.37436762225969644

In [18]:
# save RF model
pickle.dump(rf, open(base_path+"\\data-scientist-technical-test-main\\data\\auto-insurance-fall-2017\\models\\RF.pkl", 'wb'))

## XGBoost

In [19]:
# create pipeline object to standardize data for XGB

pipe = Pipeline(steps=[('XGB', XGBClassifier())])

param_grid = dict(XGB__max_depth = [5,7],
                  XGB__n_estimators = [100,150,200],
                  XGB__eta = [0.1,0.2],
                  XGB__subsample = [0.9],
                  XGB__colsample_bytree = [0.9],
                  XGB__seed = [18])

grid = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, 
                    verbose=1,
                    scoring='f1',
                    return_train_score=True)
grid.fit(X_train, y_train)
print(grid.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[10:40:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\11622563\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10:40:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\11622563\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10:40:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[10:41:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\11622563\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10:41:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\11622563\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10:41:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5305252438289872


In [20]:
#show cv results
cv_results = pd.DataFrame(grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_XGB__max_depth',
            'param_XGB__n_estimators',
            'param_XGB__eta',
            'mean_test_score',
            'std_test_score',
            'rank_test_score']].head(5)

,param_XGB__max_depth,param_XGB__n_estimators,param_XGB__eta,mean_test_score,std_test_score,rank_test_score
0,5,100,0.1,0.530525,0.009204,1
1,5,150,0.1,0.529288,0.005160,2
11,7,200,0.2,0.528039,0.009338,3
7,5,150,0.2,0.526407,0.023277,4
6,5,100,0.2,0.526218,0.015879,5


In [21]:
max_depth=cv_results[['param_XGB__max_depth', 'param_XGB__n_estimators', 'param_XGB__eta']].iloc[0,:].values[0]
n_estimators=cv_results[['param_XGB__max_depth', 'param_XGB__n_estimators', 'param_XGB__eta']].iloc[0,:].values[1]
eta=cv_results[['param_XGB__max_depth', 'param_XGB__n_estimators', 'param_XGB__eta']].iloc[0,:].values[2]
print(max_depth, n_estimators, eta)

5 100 0.1


In [22]:
xgb = XGBClassifier(objective="binary:logistic",
                    max_depth=max_depth,
                    n_estimators=n_estimators,
                    eta=eta,
                    subsample=0.9,
                    colsample_bytree=0.9,
                    seed=18).fit(X_train,y_train)

[10:41:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\11622563\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\11622563\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# f1 prediction on validation set

y_pred = xgb.predict(X_val)
f1_score(y_val, y_pred)

0.5092460881934567

In [24]:
# save XGB model
pickle.dump(xgb, open(base_path+"\\data-scientist-technical-test-main\\data\\auto-insurance-fall-2017\\models\\XGB.pkl", 'wb'))

## catboost

In [25]:
# create pipeline object to standardize data for catboost

pipe = Pipeline(steps=[('CBC', CatBoostClassifier())])

param_grid = dict(CBC__iterations=[50,100,200],
                  CBC__learning_rate=[0.1,0.2],
                  CBC__depth=[5,7])

grid = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, 
                    verbose=1,
                    scoring='f1',
                    return_train_score=True)
grid.fit(X_train, y_train)
print(grid.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
0:	learn: 0.6497193	total: 193ms	remaining: 9.44s
1:	learn: 0.6231944	total: 195ms	remaining: 4.69s
2:	learn: 0.5951020	total: 198ms	remaining: 3.11s
3:	learn: 0.5716227	total: 201ms	remaining: 2.31s
4:	learn: 0.5556958	total: 204ms	remaining: 1.83s
5:	learn: 0.5433489	total: 206ms	remaining: 1.51s
6:	learn: 0.5313765	total: 209ms	remaining: 1.29s
7:	learn: 0.5233164	total: 212ms	remaining: 1.11s
8:	learn: 0.5130638	total: 215ms	remaining: 978ms
9:	learn: 0.5072025	total: 217ms	remaining: 869ms
10:	learn: 0.4987932	total: 220ms	remaining: 780ms
11:	learn: 0.4928510	total: 223ms	remaining: 705ms
12:	learn: 0.4885225	total: 225ms	remaining: 641ms
13:	learn: 0.4823248	total: 228ms	remaining: 587ms
14:	learn: 0.4788063	total: 231ms	remaining: 539ms
15:	learn: 0.4747928	total: 234ms	remaining: 497ms
16:	learn: 0.4706300	total: 237ms	remaining: 459ms
17:	learn: 0.4687911	total: 240ms	remaining: 426ms
18:	learn: 0.4664465	total: 243

0:	learn: 0.6488232	total: 2.84ms	remaining: 139ms
1:	learn: 0.6148890	total: 5.71ms	remaining: 137ms
2:	learn: 0.5866140	total: 8.41ms	remaining: 132ms
3:	learn: 0.5649986	total: 11.3ms	remaining: 130ms
4:	learn: 0.5487792	total: 14.2ms	remaining: 128ms
5:	learn: 0.5368463	total: 16.9ms	remaining: 124ms
6:	learn: 0.5254272	total: 19.8ms	remaining: 121ms
7:	learn: 0.5153264	total: 23ms	remaining: 121ms
8:	learn: 0.5076253	total: 25.8ms	remaining: 118ms
9:	learn: 0.4996782	total: 28.4ms	remaining: 114ms
10:	learn: 0.4935675	total: 31.2ms	remaining: 110ms
11:	learn: 0.4888314	total: 34.2ms	remaining: 108ms
12:	learn: 0.4840062	total: 36.9ms	remaining: 105ms
13:	learn: 0.4790388	total: 40.3ms	remaining: 104ms
14:	learn: 0.4746411	total: 42.9ms	remaining: 100ms
15:	learn: 0.4716862	total: 45.7ms	remaining: 97.2ms
16:	learn: 0.4685821	total: 48.6ms	remaining: 94.4ms
17:	learn: 0.4655170	total: 51.4ms	remaining: 91.4ms
18:	learn: 0.4633164	total: 54.2ms	remaining: 88.4ms
19:	learn: 0.4606946

0:	learn: 0.6164030	total: 24.3ms	remaining: 1.19s
1:	learn: 0.5756661	total: 27ms	remaining: 649ms
2:	learn: 0.5378346	total: 29.9ms	remaining: 469ms
3:	learn: 0.5173478	total: 33ms	remaining: 379ms
4:	learn: 0.5027173	total: 35.7ms	remaining: 322ms
5:	learn: 0.4917794	total: 38.9ms	remaining: 285ms
6:	learn: 0.4812817	total: 41.8ms	remaining: 257ms
7:	learn: 0.4739775	total: 44.8ms	remaining: 235ms
8:	learn: 0.4673230	total: 47.5ms	remaining: 217ms
9:	learn: 0.4615497	total: 50.3ms	remaining: 201ms
10:	learn: 0.4578792	total: 52.5ms	remaining: 186ms
11:	learn: 0.4535280	total: 55ms	remaining: 174ms
12:	learn: 0.4499023	total: 57.7ms	remaining: 164ms
13:	learn: 0.4468504	total: 60.4ms	remaining: 155ms
14:	learn: 0.4440289	total: 62.8ms	remaining: 147ms
15:	learn: 0.4410261	total: 65.1ms	remaining: 138ms
16:	learn: 0.4387712	total: 66.9ms	remaining: 130ms
17:	learn: 0.4365884	total: 69.2ms	remaining: 123ms
18:	learn: 0.4343180	total: 71.4ms	remaining: 116ms
19:	learn: 0.4327992	total: 

59:	learn: 0.4103595	total: 184ms	remaining: 123ms
60:	learn: 0.4096665	total: 186ms	remaining: 119ms
61:	learn: 0.4089715	total: 189ms	remaining: 116ms
62:	learn: 0.4078111	total: 191ms	remaining: 112ms
63:	learn: 0.4070686	total: 194ms	remaining: 109ms
64:	learn: 0.4065123	total: 197ms	remaining: 106ms
65:	learn: 0.4058473	total: 200ms	remaining: 103ms
66:	learn: 0.4048871	total: 202ms	remaining: 99.5ms
67:	learn: 0.4040693	total: 204ms	remaining: 96.1ms
68:	learn: 0.4034013	total: 207ms	remaining: 92.8ms
69:	learn: 0.4029418	total: 209ms	remaining: 89.6ms
70:	learn: 0.4019072	total: 211ms	remaining: 86.4ms
71:	learn: 0.4012385	total: 214ms	remaining: 83.2ms
72:	learn: 0.4007527	total: 217ms	remaining: 80.4ms
73:	learn: 0.4002502	total: 220ms	remaining: 77.3ms
74:	learn: 0.3991434	total: 222ms	remaining: 74.2ms
75:	learn: 0.3982996	total: 225ms	remaining: 71ms
76:	learn: 0.3974340	total: 227ms	remaining: 67.9ms
77:	learn: 0.3967439	total: 230ms	remaining: 64.8ms
78:	learn: 0.3962102	

62:	learn: 0.4089562	total: 195ms	remaining: 115ms
63:	learn: 0.4081592	total: 198ms	remaining: 111ms
64:	learn: 0.4071909	total: 201ms	remaining: 108ms
65:	learn: 0.4065002	total: 203ms	remaining: 105ms
66:	learn: 0.4055765	total: 207ms	remaining: 102ms
67:	learn: 0.4049610	total: 210ms	remaining: 98.7ms
68:	learn: 0.4038664	total: 212ms	remaining: 95.4ms
69:	learn: 0.4031774	total: 215ms	remaining: 92ms
70:	learn: 0.4026106	total: 218ms	remaining: 88.8ms
71:	learn: 0.4016690	total: 220ms	remaining: 85.6ms
72:	learn: 0.4010413	total: 223ms	remaining: 82.3ms
73:	learn: 0.4002834	total: 225ms	remaining: 79ms
74:	learn: 0.3998651	total: 228ms	remaining: 75.9ms
75:	learn: 0.3992418	total: 230ms	remaining: 72.7ms
76:	learn: 0.3985394	total: 233ms	remaining: 69.5ms
77:	learn: 0.3981837	total: 235ms	remaining: 66.4ms
78:	learn: 0.3974225	total: 238ms	remaining: 63.2ms
79:	learn: 0.3963844	total: 240ms	remaining: 60.1ms
80:	learn: 0.3956024	total: 243ms	remaining: 57ms
81:	learn: 0.3950321	to

...

44:	learn: 0.3244105	total: 179ms	remaining: 617ms
45:	learn: 0.3217074	total: 183ms	remaining: 614ms
46:	learn: 0.3198171	total: 189ms	remaining: 616ms
47:	learn: 0.3176503	total: 194ms	remaining: 613ms
48:	learn: 0.3171540	total: 198ms	remaining: 611ms
49:	learn: 0.3145144	total: 203ms	remaining: 608ms
50:	learn: 0.3136092	total: 207ms	remaining: 605ms
51:	learn: 0.3119154	total: 211ms	remaining: 601ms
52:	learn: 0.3092396	total: 215ms	remaining: 597ms
53:	learn: 0.3060407	total: 219ms	remaining: 592ms
54:	learn: 0.3028852	total: 223ms	remaining: 588ms
55:	learn: 0.3002557	total: 227ms	remaining: 584ms
56:	learn: 0.2983626	total: 231ms	remaining: 579ms
57:	learn: 0.2963675	total: 235ms	remaining: 575ms
58:	learn: 0.2943094	total: 240ms	remaining: 575ms
59:	learn: 0.2919092	total: 244ms	remaining: 569ms
60:	learn: 0.2905054	total: 248ms	remaining: 564ms
61:	learn: 0.2883949	total: 251ms	remaining: 559ms
62:	learn: 0.2883273	total: 253ms	remaining: 551ms
63:	learn: 0.2851632	total: 257

44:	learn: 0.3238991	total: 179ms	remaining: 616ms
45:	learn: 0.3224589	total: 183ms	remaining: 614ms
46:	learn: 0.3190165	total: 188ms	remaining: 613ms
47:	learn: 0.3167418	total: 193ms	remaining: 611ms
48:	learn: 0.3151569	total: 197ms	remaining: 608ms
49:	learn: 0.3121792	total: 201ms	remaining: 603ms
50:	learn: 0.3093054	total: 206ms	remaining: 601ms
51:	learn: 0.3049429	total: 209ms	remaining: 596ms
52:	learn: 0.3036810	total: 213ms	remaining: 592ms
53:	learn: 0.3015680	total: 217ms	remaining: 587ms
54:	learn: 0.2989613	total: 221ms	remaining: 583ms
55:	learn: 0.2974885	total: 225ms	remaining: 579ms
56:	learn: 0.2954252	total: 229ms	remaining: 574ms
57:	learn: 0.2929619	total: 233ms	remaining: 571ms
58:	learn: 0.2914929	total: 237ms	remaining: 566ms
59:	learn: 0.2896969	total: 241ms	remaining: 562ms
60:	learn: 0.2882099	total: 245ms	remaining: 558ms
61:	learn: 0.2842888	total: 249ms	remaining: 554ms
62:	learn: 0.2822029	total: 253ms	remaining: 549ms
63:	learn: 0.2799986	total: 257

42:	learn: 0.3289905	total: 177ms	remaining: 645ms
43:	learn: 0.3255446	total: 182ms	remaining: 646ms
44:	learn: 0.3221158	total: 187ms	remaining: 644ms
45:	learn: 0.3199260	total: 192ms	remaining: 641ms
46:	learn: 0.3174563	total: 196ms	remaining: 639ms
47:	learn: 0.3160691	total: 200ms	remaining: 635ms
48:	learn: 0.3123319	total: 204ms	remaining: 629ms
49:	learn: 0.3099985	total: 208ms	remaining: 624ms
50:	learn: 0.3086115	total: 212ms	remaining: 620ms
51:	learn: 0.3062997	total: 216ms	remaining: 615ms
52:	learn: 0.3047819	total: 220ms	remaining: 610ms
53:	learn: 0.3025683	total: 224ms	remaining: 606ms
54:	learn: 0.3010612	total: 228ms	remaining: 601ms
55:	learn: 0.2987631	total: 232ms	remaining: 596ms
56:	learn: 0.2979402	total: 236ms	remaining: 591ms
57:	learn: 0.2957501	total: 240ms	remaining: 587ms
58:	learn: 0.2928932	total: 244ms	remaining: 582ms
59:	learn: 0.2902514	total: 247ms	remaining: 577ms
60:	learn: 0.2884962	total: 251ms	remaining: 572ms
61:	learn: 0.2861589	total: 255

21:	learn: 0.3775434	total: 196ms	remaining: 1.58s
22:	learn: 0.3736194	total: 203ms	remaining: 1.56s
23:	learn: 0.3702301	total: 214ms	remaining: 1.57s
24:	learn: 0.3676577	total: 220ms	remaining: 1.54s
25:	learn: 0.3647700	total: 227ms	remaining: 1.52s
26:	learn: 0.3609241	total: 234ms	remaining: 1.5s
27:	learn: 0.3591624	total: 239ms	remaining: 1.47s
28:	learn: 0.3572506	total: 247ms	remaining: 1.46s
29:	learn: 0.3550727	total: 252ms	remaining: 1.43s
30:	learn: 0.3535854	total: 258ms	remaining: 1.41s
31:	learn: 0.3518483	total: 264ms	remaining: 1.39s
32:	learn: 0.3503017	total: 270ms	remaining: 1.37s
33:	learn: 0.3487469	total: 275ms	remaining: 1.34s
34:	learn: 0.3454401	total: 280ms	remaining: 1.32s
35:	learn: 0.3442825	total: 286ms	remaining: 1.3s
36:	learn: 0.3426859	total: 294ms	remaining: 1.29s
37:	learn: 0.3399052	total: 299ms	remaining: 1.27s
38:	learn: 0.3371946	total: 309ms	remaining: 1.27s
39:	learn: 0.3359756	total: 313ms	remaining: 1.25s
40:	learn: 0.3338877	total: 318ms

0:	learn: 0.6143419	total: 2.59ms	remaining: 257ms
1:	learn: 0.5745448	total: 5.13ms	remaining: 251ms
2:	learn: 0.5350493	total: 7.85ms	remaining: 254ms
3:	learn: 0.5119854	total: 10.5ms	remaining: 252ms
4:	learn: 0.4989652	total: 13.5ms	remaining: 256ms
5:	learn: 0.4887748	total: 16.1ms	remaining: 253ms
6:	learn: 0.4757730	total: 18.8ms	remaining: 250ms
7:	learn: 0.4703294	total: 21.7ms	remaining: 250ms
8:	learn: 0.4653450	total: 24.6ms	remaining: 248ms
9:	learn: 0.4604409	total: 27.3ms	remaining: 246ms
10:	learn: 0.4548596	total: 30.1ms	remaining: 244ms
11:	learn: 0.4503120	total: 33.3ms	remaining: 244ms
12:	learn: 0.4470121	total: 36ms	remaining: 241ms
13:	learn: 0.4438680	total: 38.7ms	remaining: 238ms
14:	learn: 0.4404521	total: 41.7ms	remaining: 237ms
15:	learn: 0.4382605	total: 44.4ms	remaining: 233ms
16:	learn: 0.4359711	total: 47.2ms	remaining: 231ms
17:	learn: 0.4336543	total: 50.3ms	remaining: 229ms
18:	learn: 0.4310102	total: 53.2ms	remaining: 227ms
19:	learn: 0.4292109	tot

In [26]:
#show cv results
cv_results = pd.DataFrame(grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_CBC__iterations',
            'param_CBC__learning_rate',
            'param_CBC__depth',
            'mean_test_score',
            'std_test_score',
            'rank_test_score']].head(5)

,param_CBC__iterations,param_CBC__learning_rate,param_CBC__depth,mean_test_score,std_test_score,rank_test_score
3,100,0.2,5,0.532531,0.016446,1
11,200,0.2,7,0.530823,0.008889,2
9,100,0.2,7,0.528333,0.008278,3
5,200,0.2,5,0.526400,0.021094,4
10,200,0.1,7,0.523968,0.009599,5


In [27]:
iterations=cv_results[['param_CBC__iterations', 'param_CBC__learning_rate', 'param_CBC__depth']].iloc[0,:].values[0]
learning_rate=cv_results[['param_CBC__iterations', 'param_CBC__learning_rate', 'param_CBC__depth']].iloc[0,:].values[1]
depth=cv_results[['param_CBC__iterations', 'param_CBC__learning_rate', 'param_CBC__depth']].iloc[0,:].values[2]
print(iterations, learning_rate, depth)

100 0.2 5


In [28]:
cbc = CatBoostClassifier(iterations=iterations,
                         learning_rate=learning_rate,
                         depth=depth).fit(X_train,y_train)

0:	learn: 0.6143419	total: 2.94ms	remaining: 291ms
1:	learn: 0.5745448	total: 5.49ms	remaining: 269ms
2:	learn: 0.5350493	total: 8.14ms	remaining: 263ms
3:	learn: 0.5119854	total: 10.8ms	remaining: 260ms
4:	learn: 0.4989652	total: 14ms	remaining: 267ms
5:	learn: 0.4887748	total: 16.9ms	remaining: 265ms
6:	learn: 0.4757730	total: 19.6ms	remaining: 260ms
7:	learn: 0.4703294	total: 22.3ms	remaining: 257ms
8:	learn: 0.4653450	total: 25.1ms	remaining: 254ms
9:	learn: 0.4604409	total: 27.9ms	remaining: 251ms
10:	learn: 0.4548596	total: 30.9ms	remaining: 250ms
11:	learn: 0.4503120	total: 34.1ms	remaining: 250ms
12:	learn: 0.4470121	total: 36.9ms	remaining: 247ms
13:	learn: 0.4438680	total: 39.6ms	remaining: 244ms
14:	learn: 0.4404521	total: 42.4ms	remaining: 241ms
15:	learn: 0.4382605	total: 45.2ms	remaining: 237ms
16:	learn: 0.4359711	total: 48.1ms	remaining: 235ms
17:	learn: 0.4336543	total: 52.6ms	remaining: 239ms
18:	learn: 0.4310102	total: 56.3ms	remaining: 240ms
19:	learn: 0.4292109	tot

In [29]:
# f1 prediction on validation set

y_pred = cbc.predict(X_val)
f1_score(y_val, y_pred)

0.5276595744680851

In [30]:
# save XGB model
pickle.dump(cbc, open(base_path+"\\data-scientist-technical-test-main\\data\\auto-insurance-fall-2017\\models\\CBC.pkl", 'wb'))